
# Mini-Repaso: Transformación, Limpieza y Agrupación en **pandas**

**IMT2200 — Introducción a la Ciencia de Datos**

**Ayudantía 7**



## Objetivos
- Reforzar **transformación** y **limpieza** de datos con `pandas` (sin imputación).
- Practicar **agrupación** (`groupby` / `agg`) y **uniones** (`merge`, `concat`, `join`).



## 0) Setup


In [224]:
import pandas as pd
import numpy as np



## 1) Dataset sintético para practicar

El DF está diseñado para tocar **strings**, **fechas en múltiples formatos**, **listas** y **números con separadores**.


In [225]:
import pandas as pd

df = pd.DataFrame({ 
    "id": [" 01","03","02","004","02 "],
    "nombre_completo": ["cARLos  Muñoz","  MARÍA   lopez ","José   TORRES", None,"josé  torres "],
    "emails": ["c.munoz@uc.cl; carlos@gmail.com", "maria.lopez@uc.cl", "jtorres@mail.com; J.TORRES@UC.CL", "andrea@uc.cl", None],
    "fono": ["(+56)912345678", "9 8765 1234", "fono: 987654321", "123-456-789", "(+56) 9-2222-3333"],
    # columnas nuevas que requieren normalización
    "historial_pagos": [
        [{"fecha":"10/01/2024","monto":"CLP 5000"},{"fecha":"15/02/2024","monto":"CLP 15000"}],
        [{"fecha":"20/03/2024","monto":"CLP 8500"}],
        [],
        [{"fecha":"05/04/2024","monto":"CLP 17450"},{"fecha":"01/05/2024","monto":"CLP 3000"}],
        [{"fecha":"12/06/2024","monto":"CLP 12000"}]
    ],
    "direccion": [
        {"calle":"Av. Principal 123","comuna":"Antofagasta","region":"II"},
        {"calle":"Calle Falsa 456","comuna":"Viña del Mar","region":"V"},
        {"calle":"Los Pinos 789","comuna":"Puerto Montt","region":"X"},
        {"calle":"Av. Central 321","comuna":"La Florida","region":"RM"},
        {"calle":"Diagonal 654","comuna":"Maipú","region":"RM"}
    ]
})

df


,id,nombre_completo,emails,fono,historial_pagos,direccion
0,01,cARLos Muñoz,c.munoz@uc.cl; carlos@gmail.com,(+56)912345678,"[{'fecha': '10/01/2024', 'monto': 'CLP 5000'},...","{'calle': 'Av. Principal 123', 'comuna': 'Anto..."
1,03,MARÍA lopez,maria.lopez@uc.cl,9 8765 1234,"[{'fecha': '20/03/2024', 'monto': 'CLP 8500'}]","{'calle': 'Calle Falsa 456', 'comuna': 'Viña d..."
2,02,José TORRES,jtorres@mail.com; J.TORRES@UC.CL,fono: 987654321,[],"{'calle': 'Los Pinos 789', 'comuna': 'Puerto M..."
3,004,None,andrea@uc.cl,123-456-789,"[{'fecha': '05/04/2024', 'monto': 'CLP 17450'}...","{'calle': 'Av. Central 321', 'comuna': 'La Flo..."
4,02,josé torres,None,(+56) 9-2222-3333,"[{'fecha': '12/06/2024', 'monto': 'CLP 12000'}]","{'calle': 'Diagonal 654', 'comuna': 'Maipú', '..."



## 2) Limpieza (sin imputación)

**Reglas:** quitar duplicados, eliminar/etiquetar nulos críticos, normalizar strings, descomprimir listas, estandarizar fechas, convertir tipos.


In [226]:
clean = df.copy()

In [227]:
# A) Aplanar `direccion` (dict -> columnas)
dir_norm = pd.json_normalize(clean["direccion"])
clean_cols = pd.concat(
    [clean.drop(columns=["direccion"]).reset_index(drop=True),
     dir_norm.reset_index(drop=True)],
    axis=1
)
clean_cols

,id,nombre_completo,emails,fono,historial_pagos,calle,comuna,region
0,01,cARLos Muñoz,c.munoz@uc.cl; carlos@gmail.com,(+56)912345678,"[{'fecha': '10/01/2024', 'monto': 'CLP 5000'},...",Av. Principal 123,Antofagasta,II
1,03,MARÍA lopez,maria.lopez@uc.cl,9 8765 1234,"[{'fecha': '20/03/2024', 'monto': 'CLP 8500'}]",Calle Falsa 456,Viña del Mar,V
2,02,José TORRES,jtorres@mail.com; J.TORRES@UC.CL,fono: 987654321,[],Los Pinos 789,Puerto Montt,X
3,004,None,andrea@uc.cl,123-456-789,"[{'fecha': '05/04/2024', 'monto': 'CLP 17450'}...",Av. Central 321,La Florida,RM
4,02,josé torres,None,(+56) 9-2222-3333,"[{'fecha': '12/06/2024', 'monto': 'CLP 12000'}]",Diagonal 654,Maipú,RM


In [ ]:
# B) Aplanar `historial_pagos` (lista -> filas)
pagos_exploded = clean_cols.explode("historial_pagos", ignore_index=True)
pagos_exploded = pagos_exploded.dropna(subset=["historial_pagos"])  # elimina listas vacías

pagos_norm = pd.json_normalize(pagos_exploded["historial_pagos"])
clean = pd.concat(
    [pagos_exploded.drop(columns=["historial_pagos"]).reset_index(drop=True),
     pagos_norm.reset_index(drop=True)],
    axis=1
)
clean

,id,nombre_completo,emails,fono,calle,comuna,region,fecha,monto
0,01,cARLos Muñoz,c.munoz@uc.cl; carlos@gmail.com,(+56)912345678,Av. Principal 123,Antofagasta,II,10/01/2024,CLP 5000
1,01,cARLos Muñoz,c.munoz@uc.cl; carlos@gmail.com,(+56)912345678,Av. Principal 123,Antofagasta,II,15/02/2024,CLP 15000
2,03,MARÍA lopez,maria.lopez@uc.cl,9 8765 1234,Calle Falsa 456,Viña del Mar,V,20/03/2024,CLP 8500
3,004,None,andrea@uc.cl,123-456-789,Av. Central 321,La Florida,RM,05/04/2024,CLP 17450
4,004,None,andrea@uc.cl,123-456-789,Av. Central 321,La Florida,RM,01/05/2024,CLP 3000
5,02,josé torres,None,(+56) 9-2222-3333,Diagonal 654,Maipú,RM,12/06/2024,CLP 12000


In [229]:
# 2.1 eliminar nulos críticos
clean = clean.dropna(subset=["nombre_completo"]).reset_index(drop=True)
clean

,id,nombre_completo,emails,fono,calle,comuna,region,fecha,monto
0,01,cARLos Muñoz,c.munoz@uc.cl; carlos@gmail.com,(+56)912345678,Av. Principal 123,Antofagasta,II,10/01/2024,CLP 5000
1,01,cARLos Muñoz,c.munoz@uc.cl; carlos@gmail.com,(+56)912345678,Av. Principal 123,Antofagasta,II,15/02/2024,CLP 15000
2,03,MARÍA lopez,maria.lopez@uc.cl,9 8765 1234,Calle Falsa 456,Viña del Mar,V,20/03/2024,CLP 8500
3,02,josé torres,None,(+56) 9-2222-3333,Diagonal 654,Maipú,RM,12/06/2024,CLP 12000


In [230]:
# 2.2 normalizar strings 
clean["nombre_completo"] = (clean["nombre_completo"]
    .str.strip()                      # elimina espacios al inicio y final
    .str.title()                      # pone en formato título (primera letra mayúscula)
    .str.replace(r"\s+", " ", regex=True) # reemplaza múltiples espacios por uno solo
)
# separar nombre/apellido 
clean[["nombre", "apellido"]] = clean["nombre_completo"].str.split(" ", n=1, expand=True) # Hace solo una separación # expand=True para que retorne un DataFrame
clean

,id,nombre_completo,emails,fono,calle,comuna,region,fecha,monto,nombre,apellido
0,01,Carlos Muñoz,c.munoz@uc.cl; carlos@gmail.com,(+56)912345678,Av. Principal 123,Antofagasta,II,10/01/2024,CLP 5000,Carlos,Muñoz
1,01,Carlos Muñoz,c.munoz@uc.cl; carlos@gmail.com,(+56)912345678,Av. Principal 123,Antofagasta,II,15/02/2024,CLP 15000,Carlos,Muñoz
2,03,María Lopez,maria.lopez@uc.cl,9 8765 1234,Calle Falsa 456,Viña del Mar,V,20/03/2024,CLP 8500,María,Lopez
3,02,José Torres,None,(+56) 9-2222-3333,Diagonal 654,Maipú,RM,12/06/2024,CLP 12000,José,Torres


In [231]:
# 2.3 normalizar emails -> lista y explode
clean["emails"] = clean["emails"].str.replace(" ", "", regex=False).str.lower() # eliminar espacios y pasar a minúscula
clean["emails_list"] = clean["emails"].str.split(";") # separar por ;
clean = clean.drop(columns=["emails"]).explode("emails_list", ignore_index=True).rename(columns={"emails_list":"email"}) # explotar lista de emails
clean

,id,nombre_completo,fono,calle,comuna,region,fecha,monto,nombre,apellido,email
0,01,Carlos Muñoz,(+56)912345678,Av. Principal 123,Antofagasta,II,10/01/2024,CLP 5000,Carlos,Muñoz,c.munoz@uc.cl
1,01,Carlos Muñoz,(+56)912345678,Av. Principal 123,Antofagasta,II,10/01/2024,CLP 5000,Carlos,Muñoz,carlos@gmail.com
2,01,Carlos Muñoz,(+56)912345678,Av. Principal 123,Antofagasta,II,15/02/2024,CLP 15000,Carlos,Muñoz,c.munoz@uc.cl
3,01,Carlos Muñoz,(+56)912345678,Av. Principal 123,Antofagasta,II,15/02/2024,CLP 15000,Carlos,Muñoz,carlos@gmail.com
4,03,María Lopez,9 8765 1234,Calle Falsa 456,Viña del Mar,V,20/03/2024,CLP 8500,María,Lopez,maria.lopez@uc.cl
5,02,José Torres,(+56) 9-2222-3333,Diagonal 654,Maipú,RM,12/06/2024,CLP 12000,José,Torres,None


In [232]:
# 2.4 normalizar fono (dejar solo dígitos y remover prefijo 56)
clean["fono"] = (clean["fono"].astype(str)
                          .str.replace(r"\D+", "", regex=True) # dejar solo dígitos
                          .str.replace(r"^56", "", regex=True)) # remover prefijo 56
clean

,id,nombre_completo,fono,calle,comuna,region,fecha,monto,nombre,apellido,email
0,01,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,10/01/2024,CLP 5000,Carlos,Muñoz,c.munoz@uc.cl
1,01,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,10/01/2024,CLP 5000,Carlos,Muñoz,carlos@gmail.com
2,01,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,15/02/2024,CLP 15000,Carlos,Muñoz,c.munoz@uc.cl
3,01,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,15/02/2024,CLP 15000,Carlos,Muñoz,carlos@gmail.com
4,03,María Lopez,987651234,Calle Falsa 456,Viña del Mar,V,20/03/2024,CLP 8500,María,Lopez,maria.lopez@uc.cl
5,02,José Torres,922223333,Diagonal 654,Maipú,RM,12/06/2024,CLP 12000,José,Torres,None


In [233]:
# 2.5 normalizar id → Int64 (permite NA)
clean["id"] = (clean["id"].str.strip().astype("Int64"))
clean

,id,nombre_completo,fono,calle,comuna,region,fecha,monto,nombre,apellido,email
0,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,10/01/2024,CLP 5000,Carlos,Muñoz,c.munoz@uc.cl
1,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,10/01/2024,CLP 5000,Carlos,Muñoz,carlos@gmail.com
2,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,15/02/2024,CLP 15000,Carlos,Muñoz,c.munoz@uc.cl
3,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,15/02/2024,CLP 15000,Carlos,Muñoz,carlos@gmail.com
4,3,María Lopez,987651234,Calle Falsa 456,Viña del Mar,V,20/03/2024,CLP 8500,María,Lopez,maria.lopez@uc.cl
5,2,José Torres,922223333,Diagonal 654,Maipú,RM,12/06/2024,CLP 12000,José,Torres,None


In [234]:
# Normalizar fechas con formato "%d/%m/%Y"
clean["fecha"] = pd.to_datetime(clean["fecha"], format="%d/%m/%Y", errors="coerce")
clean

,id,nombre_completo,fono,calle,comuna,region,fecha,monto,nombre,apellido,email
0,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-01-10,CLP 5000,Carlos,Muñoz,c.munoz@uc.cl
1,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-01-10,CLP 5000,Carlos,Muñoz,carlos@gmail.com
2,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-02-15,CLP 15000,Carlos,Muñoz,c.munoz@uc.cl
3,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-02-15,CLP 15000,Carlos,Muñoz,carlos@gmail.com
4,3,María Lopez,987651234,Calle Falsa 456,Viña del Mar,V,2024-03-20,CLP 8500,María,Lopez,maria.lopez@uc.cl
5,2,José Torres,922223333,Diagonal 654,Maipú,RM,2024-06-12,CLP 12000,José,Torres,None


In [235]:
# Normalizar montos (remover "CLP " y ".")
clean["monto"] = (clean["monto"].str.replace("CLP ", "", regex=False)
                          .str.replace(".", "", regex=False)
                          .str.replace(",", "", regex=False)
                          .replace("", None)
                          .astype("Int64"))
clean

,id,nombre_completo,fono,calle,comuna,region,fecha,monto,nombre,apellido,email
0,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-01-10,5000,Carlos,Muñoz,c.munoz@uc.cl
1,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-01-10,5000,Carlos,Muñoz,carlos@gmail.com
2,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-02-15,15000,Carlos,Muñoz,c.munoz@uc.cl
3,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-02-15,15000,Carlos,Muñoz,carlos@gmail.com
4,3,María Lopez,987651234,Calle Falsa 456,Viña del Mar,V,2024-03-20,8500,María,Lopez,maria.lopez@uc.cl
5,2,José Torres,922223333,Diagonal 654,Maipú,RM,2024-06-12,12000,José,Torres,None



## 3) `merge` y **cardinalidad**

In [236]:
clean

,id,nombre_completo,fono,calle,comuna,region,fecha,monto,nombre,apellido,email
0,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-01-10,5000,Carlos,Muñoz,c.munoz@uc.cl
1,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-01-10,5000,Carlos,Muñoz,carlos@gmail.com
2,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-02-15,15000,Carlos,Muñoz,c.munoz@uc.cl
3,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-02-15,15000,Carlos,Muñoz,carlos@gmail.com
4,3,María Lopez,987651234,Calle Falsa 456,Viña del Mar,V,2024-03-20,8500,María,Lopez,maria.lopez@uc.cl
5,2,José Torres,922223333,Diagonal 654,Maipú,RM,2024-06-12,12000,José,Torres,None


In [237]:
# DF de perfil por id (1-1)
perfil = pd.DataFrame({
    "id": [1,2,3],
    "es_titular": [True, False, True]
})

# DF de pagos (1-N respecto a id)
pagos = pd.DataFrame({
    "id": [1,1,2,3,3,3],
    "fecha_pago": pd.to_datetime(["2024-03-15","2024-04-01","2024-04-20","2024-05-12","2024-05-31","2024-06-10"]),
    "monto_pagado": [5000, 5000, 12500, 1500, 3000, 0]
})

display(perfil)
display(pagos)


,id,es_titular
0,1,True
1,2,False
2,3,True


,id,fecha_pago,monto_pagado
0,1,2024-03-15,5000
1,1,2024-04-01,5000
2,2,2024-04-20,12500
3,3,2024-05-12,1500
4,3,2024-05-31,3000
5,3,2024-06-10,0


In [238]:
# Imprimir solo el dia y mes
dia_mes = perfil.merge(pagos, on="id", how="left")
dia_mes["año"] = dia_mes["fecha_pago"].dt.year
dia_mes["dia_mes"]= dia_mes["fecha_pago"].dt.strftime("%d-%m")
dia_mes

,id,es_titular,fecha_pago,monto_pagado,año,dia_mes
0,1,True,2024-03-15,5000,2024,15-03
1,1,True,2024-04-01,5000,2024,01-04
2,2,False,2024-04-20,12500,2024,20-04
3,3,True,2024-05-12,1500,2024,12-05
4,3,True,2024-05-31,3000,2024,31-05
5,3,True,2024-06-10,0,2024,10-06


In [239]:
clean

,id,nombre_completo,fono,calle,comuna,region,fecha,monto,nombre,apellido,email
0,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-01-10,5000,Carlos,Muñoz,c.munoz@uc.cl
1,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-01-10,5000,Carlos,Muñoz,carlos@gmail.com
2,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-02-15,15000,Carlos,Muñoz,c.munoz@uc.cl
3,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-02-15,15000,Carlos,Muñoz,carlos@gmail.com
4,3,María Lopez,987651234,Calle Falsa 456,Viña del Mar,V,2024-03-20,8500,María,Lopez,maria.lopez@uc.cl
5,2,José Torres,922223333,Diagonal 654,Maipú,RM,2024-06-12,12000,José,Torres,None


In [240]:
# 3.1 Inner/Left/Right/Outer
m_inner = pd.merge(clean, perfil, on="id", how="inner")
m_left  = pd.merge(clean, perfil, on="id", how="left")
m_outer = pd.merge(clean, perfil, on="id", how="outer")

display(m_inner.head())
display(m_left.head())
display(m_outer.tail())


,id,nombre_completo,fono,calle,comuna,region,fecha,monto,nombre,apellido,email,es_titular
0,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-01-10,5000,Carlos,Muñoz,c.munoz@uc.cl,True
1,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-01-10,5000,Carlos,Muñoz,carlos@gmail.com,True
2,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-02-15,15000,Carlos,Muñoz,c.munoz@uc.cl,True
3,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-02-15,15000,Carlos,Muñoz,carlos@gmail.com,True
4,3,María Lopez,987651234,Calle Falsa 456,Viña del Mar,V,2024-03-20,8500,María,Lopez,maria.lopez@uc.cl,True


,id,nombre_completo,fono,calle,comuna,region,fecha,monto,nombre,apellido,email,es_titular
0,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-01-10,5000,Carlos,Muñoz,c.munoz@uc.cl,True
1,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-01-10,5000,Carlos,Muñoz,carlos@gmail.com,True
2,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-02-15,15000,Carlos,Muñoz,c.munoz@uc.cl,True
3,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-02-15,15000,Carlos,Muñoz,carlos@gmail.com,True
4,3,María Lopez,987651234,Calle Falsa 456,Viña del Mar,V,2024-03-20,8500,María,Lopez,maria.lopez@uc.cl,True


,id,nombre_completo,fono,calle,comuna,region,fecha,monto,nombre,apellido,email,es_titular
1,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-01-10,5000,Carlos,Muñoz,carlos@gmail.com,True
2,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-02-15,15000,Carlos,Muñoz,c.munoz@uc.cl,True
3,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-02-15,15000,Carlos,Muñoz,carlos@gmail.com,True
4,2,José Torres,922223333,Diagonal 654,Maipú,RM,2024-06-12,12000,José,Torres,None,False
5,3,María Lopez,987651234,Calle Falsa 456,Viña del Mar,V,2024-03-20,8500,María,Lopez,maria.lopez@uc.cl,True


In [241]:
# 3.2 1-N: cuidado con duplicados tras el merge
m_1N = pd.merge(clean, pagos, on="id", how="left")
print("Filas clean:", len(clean), "→ tras merge 1-N:", len(m_1N))
m_1N = m_1N.sort_values(["id", "fecha_pago"]).head(10)
m_1N


Filas clean: 6 → tras merge 1-N: 12


,id,nombre_completo,fono,calle,comuna,region,fecha,monto,nombre,apellido,email,fecha_pago,monto_pagado
0,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-01-10,5000,Carlos,Muñoz,c.munoz@uc.cl,2024-03-15,5000
2,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-01-10,5000,Carlos,Muñoz,carlos@gmail.com,2024-03-15,5000
4,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-02-15,15000,Carlos,Muñoz,c.munoz@uc.cl,2024-03-15,5000
6,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-02-15,15000,Carlos,Muñoz,carlos@gmail.com,2024-03-15,5000
1,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-01-10,5000,Carlos,Muñoz,c.munoz@uc.cl,2024-04-01,5000
3,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-01-10,5000,Carlos,Muñoz,carlos@gmail.com,2024-04-01,5000
5,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-02-15,15000,Carlos,Muñoz,c.munoz@uc.cl,2024-04-01,5000
7,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-02-15,15000,Carlos,Muñoz,carlos@gmail.com,2024-04-01,5000
11,2,José Torres,922223333,Diagonal 654,Maipú,RM,2024-06-12,12000,José,Torres,None,2024-04-20,12500
8,3,María Lopez,987651234,Calle Falsa 456,Viña del Mar,V,2024-03-20,8500,María,Lopez,maria.lopez@uc.cl,2024-05-12,1500



## 4) Agrupación (`groupby`/`agg`/`apply`)

Estadísticas por **id**, **región** y combinaciones.


In [242]:

# 4.1 agregaciones básicas
g1 = m_1N.groupby("id").agg(
    n_emails=("email","nunique"),
    n_pagos=("fecha_pago","count"),
    monto_total=("monto_pagado","sum"),
    monto_promedio=("monto_pagado","mean")
).reset_index()

g1


,id,n_emails,n_pagos,monto_total,monto_promedio
0,1,2,8,40000,5000.0
1,2,0,1,12500,12500.0
2,3,1,1,1500,1500.0


In [244]:

# 4.2 multi-col y funciones diferentes
g2 = (m_1N.groupby(["nombre_completo"]).agg(ingresos=("monto","sum"), 
                                            primera_atencion=("fecha","min"),
                                            ult_pago=("fecha_pago","max"),
                                            egresos = ("monto_pagado","sum")).reset_index())
g2


,nombre_completo,ingresos,primera_atencion,ult_pago,egresos
0,Carlos Muñoz,80000,2024-01-10,2024-04-01,40000
1,José Torres,12000,2024-06-12,2024-04-20,12500
2,María Lopez,8500,2024-03-20,2024-05-12,1500


In [247]:
g2["diferencia"] = g2["ingresos"] - g2["egresos"]
g2

,nombre_completo,ingresos,primera_atencion,ult_pago,egresos,diferencia
0,Carlos Muñoz,80000,2024-01-10,2024-04-01,40000,40000
1,José Torres,12000,2024-06-12,2024-04-20,12500,-500
2,María Lopez,8500,2024-03-20,2024-05-12,1500,7000


In [248]:
# 4.3 apply para encontrar todos los correos uc
def es_uc(email):
    return email.endswith("@uc.cl") if isinstance(email, str) else False

m_1N["es_uc"] = m_1N["email"].apply(es_uc)
m_1N[m_1N["es_uc"]]

,id,nombre_completo,fono,calle,comuna,region,fecha,monto,nombre,apellido,email,fecha_pago,monto_pagado,es_uc
0,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-01-10,5000,Carlos,Muñoz,c.munoz@uc.cl,2024-03-15,5000,True
4,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-02-15,15000,Carlos,Muñoz,c.munoz@uc.cl,2024-03-15,5000,True
1,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-01-10,5000,Carlos,Muñoz,c.munoz@uc.cl,2024-04-01,5000,True
5,1,Carlos Muñoz,912345678,Av. Principal 123,Antofagasta,II,2024-02-15,15000,Carlos,Muñoz,c.munoz@uc.cl,2024-04-01,5000,True
8,3,María Lopez,987651234,Calle Falsa 456,Viña del Mar,V,2024-03-20,8500,María,Lopez,maria.lopez@uc.cl,2024-05-12,1500,True
